In [ ]:
import os
import petl as etl
#import pandas as pd
import magic
import sys
import csv
from openpyxl import Workbook
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font, NamedStyle

In [ ]:
maxInt = sys.maxsize
while True:
    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

In [ ]:
#Demander le nom du fichier de données
filename = input(str('Entrez le nom du fichier'))
# dans ce cas d'exemple: PLES_ExtractionTest_20210607

In [ ]:
#Demander le type du fichier
filetype = input(str('Type de fichier MARC21, csv, texte, xml:  ')) 

In [ ]:
#Demander le séparateur
sep = input(str('Séparateur de champs à utiliser? '))

In [ ]:
#stocker le contenu du fichier
f = os.listdir("racine/source/test/" + filename)
f

In [ ]:
#if filetype == 'csv':
# a boucler !!!!!!!!
f0 = etl.fromcsv('racine/source/test/' + filename + '/' + f[0],delimiter = sep)
f1 = etl.fromcsv('racine/source/test/' + filename + '/' + f[1],delimiter = sep)
f2 = etl.fromcsv('racine/source/test/' + filename + '/' + f[2],delimiter = sep)
f3 = etl.fromcsv('racine/source/test/' + filename + '/' + f[3],delimiter = sep)
f4 = etl.fromcsv('racine/source/test/' + filename + '/' + f[4],delimiter = sep)
f5 = etl.fromcsv('racine/source/test/' + filename + '/' + f[5],delimiter = sep)
f6 = etl.fromcsv('racine/source/test/' + filename + '/' + f[6],delimiter = sep)
f7 = etl.fromcsv('racine/source/test/' + filename + '/' + f[7],delimiter = sep)
f8 = etl.fromcsv('racine/source/test/' + filename + '/' + f[8],delimiter = sep)
f9 = etl.fromcsv('racine/source/test/' + filename + '/' + f[9],delimiter = sep)
f10 = etl.fromcsv('racine/source/test/' + filename + '/' + f[10],delimiter = sep)
f11 = etl.fromcsv('racine/source/test/' + filename + '/' + f[11],delimiter = sep)

In [ ]:
#generating the file
wb = Workbook()
#Generating the first sheet
ws = wb.active
ws.title = ("Fichiers")
#Les deux premieres lignes standard
Hd0 = ["source","Portfolio"]
Hd1 = ["Fichiers","Type","Separateur","Encodage","Quantite de donnees","Donnees","Cle du fichier","Fichier a fusionner","Cle a utiliser"]
ws.append(Hd0)
ws.append(Hd1)
#1ere colonne "fichier" 
for x in range(0,len(f)) :
    for row in ws.iter_rows(min_row=x+3, min_col=1, max_col=1, max_row=x+3):
        for cell in row:
            cell.value = f[x]
x+=1    
#2eme colonne "type"
for x in range(0,len(f)) :
    for row in ws.iter_rows(min_row=x+3, min_col=2, max_col=2, max_row=x+3):
        for cell in row:
            cell.value = magic.from_buffer(f[x])
x+=1
#3eme colonne "separateur"
for x in range(0,len(f)) :
    for row in ws.iter_rows(min_row=x+3, min_col=3, max_col=3, max_row=x+3):
        for cell in row:
            cell.value = sep
x+=1
#4eme colonne "encodage"
for x in range(0,len(f)) :
    for row in ws.iter_rows(min_row=x+3, min_col=4, max_col=4, max_row=x+3):
        for cell in row:
            cell.value = 'utf-8-'
x+=1
#5eme colonne "quantité de données"
for x in range(0,len(f)) :
    for row in ws.iter_rows(min_row=x+3, min_col=5, max_col=5, max_row=x+3):
        for cell in row:
            cell.value = len(fi)
x+=1
#saving the file
wb.save("racine/sortie/test/profil.xlsx")

In [ ]:
#Generating the 2 & 3 sheet
ws2 = wb.create_sheet("Valeurs par defaut")
ws3 = wb.create_sheet("Correspondances")

ws2['A1'] = "Code régional(telephone)"
ws2['A2'] = "Province"

wb.save("racine/sortie/test/profil.xlsx")

In [ ]:
#Generating the other sheets
for i in range (0,len(f)):
    new_ws = wb.create_sheet("ws "+f[i])
wb.save("racine/sortie/test/profil.xlsx")

In [ ]:
#Remplissage des sheets créées 
Hd2 = ["Champs","Cle","Nombres de valeurs distinctes","Nombres de valeurs vides","Types de donnees","Champ Koha","Correspondance","Operateur","Valeur1","Valeur2"]

for x in range(3,len(wb.sheetnames)):
    wb.active = x
    wb.active.append(Hd2)
wb.save("racine/sortie/test/profil.xlsx")

In [ ]:
#La premiere ligne
for x in range(3,len(wb.sheetnames)):
    for i in range (0,len(f)):
        c = list(etl.header(fi))
        for row in ws.iter_rows(min_row=2, min_col=1, max_col=1, max_row=11):
            for cell in row:
                cell.value = c[i]
                
wb.save("racine/sortie/test/profil.xlsx")

In [ ]:
#Nombre de valeurs distinctes
for x in range(3,len(wb.sheetnames)):
    for i in range (0,len(f)):
        for j in range (0,l)
            c = list(etl.header(fi))
            val_dis = list(etl.valuecounter(fi,c[i]))
            for row in ws.iter_rows(min_row=2, min_col=3, max_col=3, max_row=11):
                for cell in row:
                    cell.value = val_dis[i]
wb.save("racine/sortie/test/profil.xlsx")

In [ ]:
#Nombre de valeurs vides
for x in range(3,len(wb.sheetnames)):
    for i in range (0,len(f)):
        for j in range (0,l)
            c = list(etl.header(fi))
            val_vid = list(etl.valuecounter(fi,c[i]))
            for row in ws.iter_rows(min_row=2, min_col=3, max_col=3, max_row=11):
                for cell in row:
                    cell.value = val_vid[i]
wb.save("racine/sortie/test/profil.xlsx")